In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import selenium.webdriver.support.ui as UI
import pandas as pd 
import time

In [2]:
car_link_df = pd.read_csv("csv_files/car_links.csv")
car_link_df

,Unnamed: 0,car_links
0,0,https://www.autoscout24.com/offers/audi-a3-1-6...
1,1,https://www.autoscout24.com/offers/audi-a3-1-9...
2,2,https://www.autoscout24.com/offers/audi-a4-ohn...
3,3,https://www.autoscout24.com/offers/audi-a4-1-6...
4,4,https://www.autoscout24.com/offers/audi-a4-ava...
...,...,...
616193,616193,https://www.autoscout24.com/offers/others-trop...
616194,616194,https://www.autoscout24.com/offers/others-trop...
616195,616195,https://www.autoscout24.com/offers/others-othe...
616196,616196,https://www.autoscout24.com/offers/others-othe...


In [22]:
driver = webdriver.Chrome(ChromeDriverManager().install())
all_car_info = []


for link in car_link_df.car_links[0:3]:  
    car_data = {}  
    feature_data = []
    driver.get(link.format())

    try:
        try:
            WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,'//*[@id="gdpr-consent-notice"]')))
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="save"]'))).click()
        except:
            print("not appear")
        car_image, new_data = [], []
        carname = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#__next > div > div > main > div.StageArea_container__YuNIp > div.StageArea_informationContainer__VaFP8 > div.StageArea_titleAndActionBarContainer__soRpp > div.StageTitle_container__vP9Cw > h1"))).text
        carname = " ".join(carname.lower().split("\n"))
        car_data["name"] = carname
        carprice = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#__next > div > div > main > div.StageArea_container__YuNIp > div.StageArea_informationContainer__VaFP8 > div.Price_mainPriceContainer__syzQE > div:nth-child(1) > div.PriceInfo_styledPriceRow__2fvRD > div > span"))).text
        car_data["price"], car_data["VAT"] = carprice.split("-")

        
        time.sleep(5)
        imgs_div = driver.find_elements(By.CSS_SELECTOR, "#__next > div > div > main > div.StageArea_container__YuNIp > div.Gallery_gallery__MDr8S > div > div > div.image-gallery-thumbnails-wrapper.bottom.thumbnails-swipe-horizontal > div > div > button > img")
        for elem in imgs_div:
            car_ind_link = elem.get_attribute("src") 
            car_image.append(car_ind_link)
        
        car_data["image"] = car_image

        time.sleep(5)
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, f"#__next > div > div > main > div.DetailPage_slicesContainer__wHHae > div > div > div.DetailsSection_childrenSection__NQLD7 > button"))).click()
        equipment_value = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, f"#__next > div > div > main > div.DetailPage_slicesContainer__wHHae > div > div > div.DetailsSection_childrenSection__NQLD7 > div > dl > dd > ul"))).text
        car_data["equipment"] = equipment_value.lower()
        

        contactName = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, f"#__next > div > div > main > div.DetailPage_slicesContainer__wHHae > div.VendorAndCtaSection_container__ivfyv > div > div.VendorData_mainContainer__AWyih > div.VendorData_bodyContainer__sJZZC > div.Contact_container__TYO5v > span.Contact_contactName__MFXhS"))).text
        contactNumber = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, f"#__next > div > div > main > div.DetailPage_slicesContainer__wHHae > div.VendorAndCtaSection_container__ivfyv > div > div.VendorData_mainContainer__AWyih > div.VendorData_bodyContainer__sJZZC > div.Contact_container__TYO5v > div > a"))).text
        contactAddress = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, f"#__next > div > div > main > div.DetailPage_slicesContainer__wHHae > div.VendorAndCtaSection_container__ivfyv > div > div.VendorData_mainContainer__AWyih > div.VendorData_bodyContainer__sJZZC > div.Department_openingHoursContainer__VP9fd > div.Department_departmentContainer__uYING > a > div"))).text
        companyName = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, f"#__next > div > div > main > div.DetailPage_slicesContainer__wHHae > div.VendorAndCtaSection_container__ivfyv > div > div.VendorData_mainContainer__AWyih > div.VendorData_bodyContainer__sJZZC > div.RatingsAndCompanyName_container__9_HA4 > div.RatingsAndCompanyName_dealer__HTXk_ > div:nth-child(1)"))).text
        

        car_data["contact_name"] = contactName.lower()
        car_data["contact_number"] = contactNumber.lower()
        car_data["contact_address"] = contactAddress.lower()
        car_data["company_name"] = companyName.lower()

        for i in range(0, 11):
            try:
                basic_data = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, f"#__next > div > div > main > div.DetailPage_slicesContainer__wHHae > div:nth-child({i}) > div"))).text
                feature_data.append(basic_data)
            except:
                pass
        
    

        for data in feature_data:
            if data != "":
                data = data.lower().split("\n")
                data.pop(0)
                for key, val in zip(data[::2], data[1::2]):
                    car_data[key] = val
                

    except:
        pass
    
    if car_data != "":
        all_car_info.append(car_data)

driver.quit() 
all_car_info    

/tmp/ipykernel_15885/803424496.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


not appear
not appear


[{'name': 'audi a3 1.6 klimaautomatik+el.fensterheber',
  'price': '€ 699.',
  'VAT': '',
  'image': ['https://prod.pictures.autoscout24.net/listing-images/a3b26e17-6ee7-4191-a89e-bec57c3599f8_bb777f51-7ea4-4e45-9ff7-05d4fd4532af.jpg/120x90.jpg',
   'https://prod.pictures.autoscout24.net/listing-images/a3b26e17-6ee7-4191-a89e-bec57c3599f8_3864709d-9214-4df3-a2e2-1cde37a9edf4.jpg/120x90.jpg',
   'https://prod.pictures.autoscout24.net/listing-images/a3b26e17-6ee7-4191-a89e-bec57c3599f8_6caacbf5-e630-4a20-b6d8-1bed5a58ba5e.jpg/120x90.jpg',
   'https://prod.pictures.autoscout24.net/listing-images/a3b26e17-6ee7-4191-a89e-bec57c3599f8_55cefa3c-558f-4858-a8b5-75dfe1209d5e.jpg/120x90.jpg',
   'https://prod.pictures.autoscout24.net/listing-images/a3b26e17-6ee7-4191-a89e-bec57c3599f8_55b57962-9ab9-40e1-b1c6-342c9d19982a.jpg/120x90.jpg',
   'https://prod.pictures.autoscout24.net/listing-images/a3b26e17-6ee7-4191-a89e-bec57c3599f8_0d1d0845-baad-4abd-88ff-a762c73b2a38.jpg/120x90.jpg',
   'https://p

In [119]:
_df = pd.DataFrame(all_car_info) 

In [120]:
_df

,name,price,VAT,image,equipment,contact_name,contact_number,contact_address,company_name,body type,...,on-board computer,central door lock with remote control,all season tyres,paint,in dieser anzeige bieten wir einen audi a4 tdi zum verkauf an.,general inspection,6.5 l/100 km (city),power windows,side airbag,alloy wheels
0,audi a3 1.6 klimaautomatik+el.fensterheber,€ 699.,,[https://prod.pictures.autoscout24.net/listing...,air conditioning\nautomatic climate control\ne...,andreas astaller,+49 (0)9451 - 8333000,"dieselstr. 1,\n84069 schierling, de",autohaus astaller gmbh,sedan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,audi a4 ohne tüv,€ 390.,,[https://prod.pictures.autoscout24.net/listing...,air conditioning\nautomatic climate control\np...,team verkauf,+49 (0)5365 - 96062,"neue straße 53,\n38444 wolfsburg heiligendorf, de",hoffmann automobile,sedan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,audi a4 avant 1.9 tdi 110cv,€ 400.,,[https://prod.pictures.autoscout24.net/listing...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"audi a4 avant 1.9 tdi, klimaautomatik, automat...","€ 1,000.",,[https://prod.pictures.autoscout24.net/listing...,air conditioning\narmrest\nautomatic climate c...,herr hennig,+49 (0)37296 - 886100,"stollberger str. 19,\n09387 jahnsdorf/pfaffenh...",ps-automobile,station wagon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"audi a6 avant 2,5 tdi",€ 500.,,[https://prod.pictures.autoscout24.net/listing...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,audi a4 avant 1.9 tdi,€ 500.,,[https://prod.pictures.autoscout24.net/listing...,air conditioning,axel janaschke,+49 (0)3947 - 7734000,"badeborner weg 2d,\n06484 quedlinburg, de",automietkauf quedlinburg,station wagon,...,radio,driver-side airbag,catalytic converter,metallic,das fahrzeug wird nur an händler oder gewerbet...,NaN,NaN,NaN,NaN,NaN
9,audi a3 1.9 tdi ambiente,€ 900.,,[https://prod.pictures.autoscout24.net/listing...,air conditioning\narmrest\nautomatic climate c...,lulzim hajdini,+49 (0)8121 - 7054008,"gruber str. 62,\n85586 poing, de",auto luli,sedan,...,NaN,NaN,NaN,NaN,NaN,11/2022,4.1 l/100 km (country),safety & security,extras,summer tyres


In [126]:
# read_df = pd.read_csv("csv_files/scrape_data.csv")
# read_df.head(5)

read_df = _df.copy()
read_df

,name,price,VAT,image,equipment,contact_name,contact_number,contact_address,company_name,body type,...,on-board computer,central door lock with remote control,all season tyres,paint,in dieser anzeige bieten wir einen audi a4 tdi zum verkauf an.,general inspection,6.5 l/100 km (city),power windows,side airbag,alloy wheels
0,audi a3 1.6 klimaautomatik+el.fensterheber,€ 699.,,[https://prod.pictures.autoscout24.net/listing...,air conditioning\nautomatic climate control\ne...,andreas astaller,+49 (0)9451 - 8333000,"dieselstr. 1,\n84069 schierling, de",autohaus astaller gmbh,sedan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,audi a4 ohne tüv,€ 390.,,[https://prod.pictures.autoscout24.net/listing...,air conditioning\nautomatic climate control\np...,team verkauf,+49 (0)5365 - 96062,"neue straße 53,\n38444 wolfsburg heiligendorf, de",hoffmann automobile,sedan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,audi a4 avant 1.9 tdi 110cv,€ 400.,,[https://prod.pictures.autoscout24.net/listing...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"audi a4 avant 1.9 tdi, klimaautomatik, automat...","€ 1,000.",,[https://prod.pictures.autoscout24.net/listing...,air conditioning\narmrest\nautomatic climate c...,herr hennig,+49 (0)37296 - 886100,"stollberger str. 19,\n09387 jahnsdorf/pfaffenh...",ps-automobile,station wagon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"audi a6 avant 2,5 tdi",€ 500.,,[https://prod.pictures.autoscout24.net/listing...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,audi a4 avant 1.9 tdi,€ 500.,,[https://prod.pictures.autoscout24.net/listing...,air conditioning,axel janaschke,+49 (0)3947 - 7734000,"badeborner weg 2d,\n06484 quedlinburg, de",automietkauf quedlinburg,station wagon,...,radio,driver-side airbag,catalytic converter,metallic,das fahrzeug wird nur an händler oder gewerbet...,NaN,NaN,NaN,NaN,NaN
9,audi a3 1.9 tdi ambiente,€ 900.,,[https://prod.pictures.autoscout24.net/listing...,air conditioning\narmrest\nautomatic climate c...,lulzim hajdini,+49 (0)8121 - 7054008,"gruber str. 62,\n85586 poing, de",auto luli,sedan,...,NaN,NaN,NaN,NaN,NaN,11/2022,4.1 l/100 km (country),safety & security,extras,summer tyres


In [127]:

read_df = read_df[read_df["name"].notnull()]
read_df

,name,price,VAT,image,equipment,contact_name,contact_number,contact_address,company_name,body type,...,on-board computer,central door lock with remote control,all season tyres,paint,in dieser anzeige bieten wir einen audi a4 tdi zum verkauf an.,general inspection,6.5 l/100 km (city),power windows,side airbag,alloy wheels
0,audi a3 1.6 klimaautomatik+el.fensterheber,€ 699.,,[https://prod.pictures.autoscout24.net/listing...,air conditioning\nautomatic climate control\ne...,andreas astaller,+49 (0)9451 - 8333000,"dieselstr. 1,\n84069 schierling, de",autohaus astaller gmbh,sedan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,audi a4 ohne tüv,€ 390.,,[https://prod.pictures.autoscout24.net/listing...,air conditioning\nautomatic climate control\np...,team verkauf,+49 (0)5365 - 96062,"neue straße 53,\n38444 wolfsburg heiligendorf, de",hoffmann automobile,sedan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,audi a4 avant 1.9 tdi 110cv,€ 400.,,[https://prod.pictures.autoscout24.net/listing...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"audi a4 avant 1.9 tdi, klimaautomatik, automat...","€ 1,000.",,[https://prod.pictures.autoscout24.net/listing...,air conditioning\narmrest\nautomatic climate c...,herr hennig,+49 (0)37296 - 886100,"stollberger str. 19,\n09387 jahnsdorf/pfaffenh...",ps-automobile,station wagon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"audi a6 avant 2,5 tdi",€ 500.,,[https://prod.pictures.autoscout24.net/listing...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,audi a4 avant 1.9 tdi,€ 500.,,[https://prod.pictures.autoscout24.net/listing...,air conditioning,axel janaschke,+49 (0)3947 - 7734000,"badeborner weg 2d,\n06484 quedlinburg, de",automietkauf quedlinburg,station wagon,...,radio,driver-side airbag,catalytic converter,metallic,das fahrzeug wird nur an händler oder gewerbet...,NaN,NaN,NaN,NaN,NaN
9,audi a3 1.9 tdi ambiente,€ 900.,,[https://prod.pictures.autoscout24.net/listing...,air conditioning\narmrest\nautomatic climate c...,lulzim hajdini,+49 (0)8121 - 7054008,"gruber str. 62,\n85586 poing, de",auto luli,sedan,...,NaN,NaN,NaN,NaN,NaN,11/2022,4.1 l/100 km (country),safety & security,extras,summer tyres


In [128]:
read_df.head()

,name,price,VAT,image,equipment,contact_name,contact_number,contact_address,company_name,body type,...,on-board computer,central door lock with remote control,all season tyres,paint,in dieser anzeige bieten wir einen audi a4 tdi zum verkauf an.,general inspection,6.5 l/100 km (city),power windows,side airbag,alloy wheels
0,audi a3 1.6 klimaautomatik+el.fensterheber,€ 699.,,[https://prod.pictures.autoscout24.net/listing...,air conditioning\nautomatic climate control\ne...,andreas astaller,+49 (0)9451 - 8333000,"dieselstr. 1,\n84069 schierling, de",autohaus astaller gmbh,sedan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,audi a4 ohne tüv,€ 390.,,[https://prod.pictures.autoscout24.net/listing...,air conditioning\nautomatic climate control\np...,team verkauf,+49 (0)5365 - 96062,"neue straße 53,\n38444 wolfsburg heiligendorf, de",hoffmann automobile,sedan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,audi a4 avant 1.9 tdi 110cv,€ 400.,,[https://prod.pictures.autoscout24.net/listing...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"audi a4 avant 1.9 tdi, klimaautomatik, automat...","€ 1,000.",,[https://prod.pictures.autoscout24.net/listing...,air conditioning\narmrest\nautomatic climate c...,herr hennig,+49 (0)37296 - 886100,"stollberger str. 19,\n09387 jahnsdorf/pfaffenh...",ps-automobile,station wagon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"audi a6 avant 2,5 tdi",€ 500.,,[https://prod.pictures.autoscout24.net/listing...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
read_df['price'] = read_df['price'].str.replace(r'€', '')
read_df['price'] = read_df['price'].str.replace(r',', '')
read_df['price'] = read_df['price'].str.replace(r'.', '')
read_df['price'] = read_df['price'].str.replace(r' ', '')
read_df['price']

/tmp/ipykernel_10458/2579416578.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  read_df['price'] = read_df['price'].str.replace(r'€', '')
/tmp/ipykernel_10458/2579416578.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  read_df['price'] = read_df['price'].str.replace(r',', '')
/tmp/ipykernel_10458/2579416578.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when 

0     699
2     390
4     400
6    1000
7     500
8     500
9     900
Name: price, dtype: object

In [130]:
kw_power = []
for power in read_df["power"]:
    if power != "":
        if isinstance(power, str):
            p, _, _, _= power.split(" ")
            kw_power.append(p)
        else:
            kw_power.append(power)
    else:
        kw_power.append("")

read_df["power"] = kw_power

/tmp/ipykernel_10458/2441152756.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  read_df["power"] = kw_power


In [131]:
read_df['mileage'] = read_df['mileage'].str.replace(r'km', '')
read_df['mileage'] = read_df['mileage'].str.replace(r',', '')
read_df['mileage'] = read_df['mileage'].str.replace(r' ', '')
read_df['mileage'][0]

/tmp/ipykernel_10458/2843263789.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  read_df['mileage'] = read_df['mileage'].str.replace(r'km', '')
/tmp/ipykernel_10458/2843263789.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  read_df['mileage'] = read_df['mileage'].str.replace(r',', '')
/tmp/ipykernel_10458/2843263789.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

'360360'

In [132]:
fr_year = []
for year in read_df["first registration"]:
    if isinstance(year, str):
        _, p = year.split("/")
        fr_year.append(p)
    else:
        fr_year.append(year)
read_df["first registration"] = fr_year

/tmp/ipykernel_10458/3199899734.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  read_df["first registration"] = fr_year


In [133]:
read_df['equipment'] = read_df['equipment'].str.split('\n')
read_df['equipment']

/tmp/ipykernel_10458/614429379.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  read_df['equipment'] = read_df['equipment'].str.split('\n')


0    [air conditioning, automatic climate control, ...
2    [air conditioning, automatic climate control, ...
4                                                  NaN
6    [air conditioning, armrest, automatic climate ...
7                                                  NaN
8                                   [air conditioning]
9    [air conditioning, armrest, automatic climate ...
Name: equipment, dtype: object

In [134]:
read_df

,name,price,VAT,image,equipment,contact_name,contact_number,contact_address,company_name,body type,...,on-board computer,central door lock with remote control,all season tyres,paint,in dieser anzeige bieten wir einen audi a4 tdi zum verkauf an.,general inspection,6.5 l/100 km (city),power windows,side airbag,alloy wheels
0,audi a3 1.6 klimaautomatik+el.fensterheber,699,,[https://prod.pictures.autoscout24.net/listing...,"[air conditioning, automatic climate control, ...",andreas astaller,+49 (0)9451 - 8333000,"dieselstr. 1,\n84069 schierling, de",autohaus astaller gmbh,sedan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,audi a4 ohne tüv,390,,[https://prod.pictures.autoscout24.net/listing...,"[air conditioning, automatic climate control, ...",team verkauf,+49 (0)5365 - 96062,"neue straße 53,\n38444 wolfsburg heiligendorf, de",hoffmann automobile,sedan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,audi a4 avant 1.9 tdi 110cv,400,,[https://prod.pictures.autoscout24.net/listing...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"audi a4 avant 1.9 tdi, klimaautomatik, automat...",1000,,[https://prod.pictures.autoscout24.net/listing...,"[air conditioning, armrest, automatic climate ...",herr hennig,+49 (0)37296 - 886100,"stollberger str. 19,\n09387 jahnsdorf/pfaffenh...",ps-automobile,station wagon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"audi a6 avant 2,5 tdi",500,,[https://prod.pictures.autoscout24.net/listing...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,audi a4 avant 1.9 tdi,500,,[https://prod.pictures.autoscout24.net/listing...,[air conditioning],axel janaschke,+49 (0)3947 - 7734000,"badeborner weg 2d,\n06484 quedlinburg, de",automietkauf quedlinburg,station wagon,...,radio,driver-side airbag,catalytic converter,metallic,das fahrzeug wird nur an händler oder gewerbet...,NaN,NaN,NaN,NaN,NaN
9,audi a3 1.9 tdi ambiente,900,,[https://prod.pictures.autoscout24.net/listing...,"[air conditioning, armrest, automatic climate ...",lulzim hajdini,+49 (0)8121 - 7054008,"gruber str. 62,\n85586 poing, de",auto luli,sedan,...,NaN,NaN,NaN,NaN,NaN,11/2022,4.1 l/100 km (country),safety & security,extras,summer tyres


In [135]:
read_df.to_csv("csv_files/test_data.csv")
# read_df.head(5)